## Les Miserables Network Analysis 

### Set up environment 

In [13]:
# set up environment
import json 
import altair as alt
import networkx as nx
import nx_altair as nxa

### Load data 

To begin our analysis, we need to load in a network dataset from a json file. NetworkX does not load and parse JSON transparently so we need the json package. 

In [3]:
# load file 
with open('miserables.json') as mis:
    les_mis = json.load(mis)

In [4]:
# pass dataset, not a multigraph 
G = nx.readwrite.json_graph.node_link_graph(les_mis, multigraph=False) 

nxa.draw_networkx(G, node_tooltip='name')

alt.LayerChart(...)

### Data exploration 
Just like with any dataframe, we need to do some exploratory analysis: 

In [12]:
# pull node and edge count 
print(nx.info(G))

Graph with 77 nodes and 254 edges


### Density
Another metric we can use to explore our data is density. Density ranges from 0-1 with a value of zero meaning no connections and a value of one meaning that every node is connected to every other node. It can also relate to the probablity of two random nodes being connected. 

In [14]:
density = nx.density(G)
print('Network density:', density)

Network density: 0.08680792891319207


### Shortest Path
Aside from network density, the shortest path between a pair of nodes is also valuable information in a network analysis. In this case, we can find the shortest connection between two characters: 

In [15]:
# get ids
names = ('Napoleon', 'Jondrette')

ids = [x for x,y in G.nodes(data=True) if y['name'] in names]
ids

[1, 46]

In [22]:
# get shortest path ids
path = nx.shortest_path(G, source=ids[0], target=ids[1])

print('Shortest path between {} and {}:'.format(names[0], names[1]), path)

Shortest path between Napoleon and Jondrette: [1, 0, 11, 48, 47, 46]


In [23]:
# convert ids to name
names = [G.nodes[id]['name'] for id in path] 
names

['Napoleon', 'Myriel', 'Valjean', 'Gavroche', 'Mme.Burgon', 'Jondrette']

#### Length of Path 
The list above gives us the shortest path from Napoleon to Jondrette. To find the length of the path, we can take the number of elements in the list and subtract 1: 

In [24]:
print('Length of path:', (len(path)-1))

Length of path: 5


### Measures of Centrality

#### Degrees

Another imporant measure is a node's degree, which is the number of connections a single node has to other nodes. Here, we can find each individual node's degree as well as save it as a node attribute: 

In [33]:
# get degrees 
degrees = dict(G.degree(G.nodes()))

# save degrees as a node attribute 
nx.set_node_attributes(G, degrees, 'degree')

In [36]:
# confirm degree attribute 
G.nodes.data()[0]

{'name': 'Myriel', 'group': 1, 'degree': 10}

Now that our node degree information is complete, we can find other useful insights about our data such as connectivity. In our case, we can find which characters had the most encounters with other characters: 

In [38]:
# reverse sort by degrees 
sorted_degrees = sorted(degrees.items(), key=lambda x : x[1], reverse=True)

In [40]:
# print top twn nodes by degree 
print('Top Ten Nodes by Degree:\n')

for degree in sorted_degrees[:10]:
    print('- {} has {} neighbors'.format(G.nodes[degree[0]]['name'], degree[1]))

Top Ten Nodes by Degree:

- Valjean has 36 neighbors
- Gavroche has 22 neighbors
- Marius has 19 neighbors
- Javert has 17 neighbors
- Thenardier has 16 neighbors
- Fantine has 15 neighbors
- Enjolras has 15 neighbors
- Courfeyrac has 13 neighbors
- Bossuet has 13 neighbors
- Bahorel has 12 neighbors


#### Betweenness Centrality

Although node degree can be quite useful, it only takes immediate neighbors into account and does not necessarily measure the importance of a node. However, betweenness centrality take into account all of the shorest paths between node pair and is a more appropriate measure to determine the importance of a node. A value of 0 means that no shortest paths go through this node while a value of 1 means that all shortest paths go through this node. 

In [49]:
# betweenness centrality 
between = nx.betweenness_centrality(G)

# set node attribute 
nx.set_node_attributes(G, between, 'between')

# sort by betweenness 
sorted_between = sorted(between.items(), key=lambda x : x[1], reverse=True)

print('Top Ten Nodes by Betweenness:\n')

for between in sorted_between[:10]: 
    print('- {}: {} '.format(G.nodes[between[0]]['name'], between[1]))

Top Ten Nodes by Betweenness:

- Valjean: 0.5699890527836184 
- Myriel: 0.17684210526315788 
- Gavroche: 0.16511250242584766 
- Marius: 0.132032488621946 
- Fantine: 0.12964454098819422 
- Thenardier: 0.07490122123424225 
- Javert: 0.05433155966478436 
- Mlle.Gillenormand: 0.047598927875243675 
- Enjolras: 0.0425533568221771 
- Tholomyes: 0.04062934817733579 


#### Eigenvetor Centrality 

Another measure of centrality includes Eigenvector centrality. This computes the centrality of a node based on the centrality of its neighbors. 

In [51]:
# get centrality 
eigen = nx.eigenvector_centrality(G)

# set node attribute
nx.set_node_attributes(G, eigen, 'centrality')

# sort by eigenvector centrality 
sorted_eigen = sorted(eigen.items(), key=lambda x : x[1], reverse=True)

print('Top Ten Nodes by Eigenvector Centrality:\n')

for e in sorted_eigen[:10]: 
    print(' - {}: {}'.format(G.nodes[e[0]]['name'], e[1]))

Top Ten Nodes by Eigenvector Centrality:

 - Gavroche: 0.31783893977497674
 - Valjean: 0.2676181759885393
 - Enjolras: 0.26717863282356663
 - Marius: 0.25911114534178753
 - Bossuet: 0.24213078637474134
 - Courfeyrac: 0.23246719717021405
 - Bahorel: 0.22155360926119963
 - Joly: 0.22155360926119963
 - Combeferre: 0.21073457488115616
 - Feuilly: 0.21073457488115616


### Data Visualization

Now that we've extracted all of these metrics from are data, we can visualize them: 

In [85]:
# compute positions 
pos = nx.spring_layout(G, iterations=100)

# selections
brush = alt.selection_interval(encodings=['x', 'y'])
multi = alt.selection(type='multi', fields=['group'])

# network chart 
chart = nxa.draw_networkx(G, pos=pos, 
    node_size = 100, 
    node_color = 'group:N', 
    width = 'value:Q', 
    node_tooltip = 'name', 
    linewidths = 0)

# node layers 
edges = chart.layer[0]
nodes = chart.layer[1]

# group values 
groups = list(range(1,10))

# color settings 
color = alt.Color('group:N', scale=alt.Scale(domain=groups), legend=None)

# node conditions
nodes = nodes.encode(
    opacity = alt.condition(brush, alt.value(1), alt.value(0.25)), 
    fill = alt.condition(multi, color, alt.value('lightgray'))
).properties(
    title='Les Miserables Network'
).add_selection(
    brush, 
    multi
)

# bar graph 
bars = alt.Chart(nodes.data).mark_bar().encode(
    x = alt.X('count()', scale=alt.Scale(domain=(0, 15))), 
    y = alt.Y('group:N', scale=alt.Scale(domain=(groups)), title='Group'), 
    color = color, 
    opacity = alt.condition(multi, alt.value(1), alt.value(0.25))
).transform_filter(
    brush
).add_selection(
    multi
) 

alt.vconcat(edges+nodes, bars)

alt.VConcatChart(...)

Our visualization above allows us to see all our characters as well as the connections between them. Additionally, our visualization allows for interactivity in both our charts. The top chart allows for interval selection across both x and y axes of our network, which is reflected on the bottom chart. The bottom chart allows for the selection of a specific group of characters which is reflected on the top chart. 